## Ideas:

### Attributes

- illumination (one or two) - how does Flamingo converter check these attributes?
- channels
- time points
- positions
- frame interval - diff between last and first time stamp.
- magnification - probably just parse metadata file.

### Methods

- Save max projection
    - Memory converving option: open/save individual frames in dir
    - Full stack option: concatenate MPs into stack and save stack.
- Downsample 3D
    - Memory converving option: open/save individual 3D frames in dir
    - Full stack option: concatenate MPs into stack and save stack.
    - View in Napari True/False
- Combined Max projection and Downsample 3D
    - Computes both at once as a nice saver. Will use slightly more memory but save a lot of time. 
    - 

### Thoughts

- it would probably be faster to 1) max project each side, then max project those together and 2) downsample 3d volumes before merging instead of the current workflow which is to merge the full volume before max projecting and merging
- measure the time of each step (reading, processing, writing); see if it's worth reading in a separate thread from processing. Might need both multithreading and multiiprocessing I suppose. 

go through the files and make a list of files for each region

go through the files in each region and 

In [15]:
import os
import sys
import numpy as np
from pathlib import Path
from datetime import date 

class Kkpo:

    def __init__(self, file_path=None):
        if file_path == None:
            print('*****'*9)
            print("I can't make a Kakapo without a file path!")
            print('*****'*9)
            sys.exit()
        self.file_path = Path(file_path)
        self.files = [file for file in os.listdir(self.file_path) if file.endswith('.tif') and not file.startswith('.') and not 'MP' in file]
        self.timepoint_names = [file.split('_')[1] for file in self.files]
        self.region_names = [file.split('_')[3] for file in self.files]
        self.channel_names = [file.split('_')[6] for file in self.files]
        self.illum_names = [file.split('_')[7] for file in self.files]

        self.num_regions = len(np.unique(self.region_names))
        self.num_channels = len(np.unique(self.channel_names))
        self.num_illum = len(np.unique(self.illum_names))
        self.num_timepoints = len(np.unique(self.timepoint_names))
        self.get_interval()

    def get_interval(self):
        setting_files = [file for file in os.listdir(self.file_path) if "Settings.txt" in file and not file.startswith('.')]
        first_timepoint_name = [file for file in setting_files if all(match in file for match in [self.timepoint_names[0], self.region_names[0], self.channel_names[0], self.illum_names[0]])]
        last_timepoint_name = [file for file in setting_files if all(match in file for match in [self.timepoint_names[-1], self.region_names[0], self.channel_names[0], self.illum_names[0]])]
        # quality control
        if len(first_timepoint_name) != 1 or len(last_timepoint_name) != 1:
            print('*****'*9)
            print(f'EEROR:Problem with the settings file!\n{len(first_timepoint_name)} first and {len(last_timepoint_name)} last timepoint names found.\n',
                  f'Expected 1 first and 1 last timepoint name. Exiting...')
            print('*****'*9)
            sys.exit()
        # read the settings file, get the start and end times
        first_timepoint_file = np.loadtxt(self.file_path / first_timepoint_name[0], dtype='str', delimiter = '/n')
        last_timepoint_file = np.loadtxt(self.file_path / last_timepoint_name[0], dtype='str', delimiter = '/n')
        first_timestamp_line = [line for line in first_timepoint_file if 'Date time stamp' in line][0]
        last_timestamp_line = [line for line in last_timepoint_file if 'Date time stamp' in line][0]
        first_timestamp_val = first_timestamp_line.split('=')[-1]
        last_timestamp_val = last_timestamp_line.split('=')[-1]
        first_timestamp_date_raw = first_timestamp_val.split('_')[0]
        last_timestamp_date_raw = last_timestamp_val.split('_')[0]

        d0 = date(int(first_timestamp_date_raw[0:5]), int(first_timestamp_date_raw[5:7]), int(first_timestamp_date_raw[7:9]))
        d1 = date(int(last_timestamp_date_raw[0:5]), int(last_timestamp_date_raw[5:7]), int(last_timestamp_date_raw[7:9]))
        first_timestamp_time = int(first_timestamp_val.split('_')[-1])
        last_timestamp_time = int(last_timestamp_val.split('_')[-1])
        print(f'first timepoint date is {first_timestamp_date_raw}')
        print(f'first timepoint time is {first_timestamp_time}')
        print(f'last timepoint date is {last_timestamp_date_raw}')
        print(f'last timepoint time is {last_timestamp_time}')
        # calculate interval from first and last timepoint
        hours = last_timestamp_time - first_timestamp_time
        print(hours)
        print(d1-d0)

    def parse_settings(self):
        pass
        
'''

convert this to min sec etc with datetime
'''

kkpo = Kkpo('E:\kkpo_test\song')

print(kkpo.num_timepoints)

first timepoint date is  20220331
first timepoint time is 190535
last timepoint date is  20220331
last timepoint time is 205534
14999
0:00:00
12


In [37]:
filepath = '/Volumes/FlashSSD2/song/S000_t000000_V000_R0002_X000_Y000_C02_I0_D0_P00294_Settings.txt'
file = np.loadtxt(filepath, dtype='str', delimiter = '/n')

print(file)

['<Workflow Settings>' '  <Experiment Settings>'
 '    Plane spacing (um) = 2.5' '    Frame rate (f/s) = 33.335556'
 '    Exposure time (us) = 29998'
 '    Duration (dd:hh:mm:ss) = 00:13:00:00'
 '    Interval (dd:hh:mm:ss) = 00:00:10:00' '    Sample = '
 '    Number of angles = ' '    Angle step size = ' '    Region = 2'
 '    Save image directory = /scratch/20220331_190533_EXP84'
 '    Comments = BestExperimentEver' '    Save max projection = false'
 '    Display max projection = false'
 '    Save image data in tiff format = true'
 '    Save image data in raw format = false'
 '    Save to subfolders = false'
 '    Work flow live view enabled = false' '  </Experiment Settings>'
 '  <Camera Settings>' '    Exposure time (us) = 29998'
 '    Frame rate (f/s) = 33.335556' '    AOI width = 2048'
 '    AOI height = 2048' '  </Camera Settings>' '  <Stack Settings>'
 '    Stack index = 2' '    Change in Z axis (mm) = 0.734'
 '    Number of planes = 294' '    Number of planes saved = 0'
 '    S

In [16]:
class Test(object):
    def __init__(self):
        self.hi()

    def hi(self):
        print ("Hi!")

In [17]:
test = Test()

Hi!


In [ ]:
file = 'S000_t000000_V000_R0000_X000_Y000_C00_I0_D0_P00135.tif'


        print(f'unique sample names: {np.unique(sample_names)}')
        #print(f'unique time point names: {np.unique(timepoint_names)}')
        print(f'unique view names: {np.unique(view_names)}')
        print(f'unique region names: {np.unique(region_names)}')
        print(f'unique tileX names: {np.unique(tileX_names)}')
        print(f'unique tileY names: {np.unique(tileY_names)}')
        print(f'unique chanel names: {np.unique(channel_names)}')
        print(f'unique illumination names: {np.unique(illum_names)}')

        ch_640 = [file for file in files if 'C00' in file]
        ch_561 = [file for file in files if 'C01' in file]
        ch_488 = [file for file in files if 'C02' in file]
        ch_405 = [file for file in files if 'C03' in file]
        trans = [file for file in files if 'C04' in file]

        print('***'*8)
        print(f'num channels is {num_channels}')
        print(f'num trans images is {len(trans)}')
        print(f'num 405 images is {len(ch_405)}')
        print(f'num 488 images is {len(ch_488)}')
        print(f'num 561 images is {len(ch_561)}')
        print(f'num 640 images is {len(ch_640)}')
        print('***'*8)